In [1]:
#start

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7,application_1727057780008_0009,pyspark,idle,Link,Link,user_n-renukumar_rajendran_draup_com,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc.addPyFile("s3://jobs-emr-etl-config/jobs_run/prod/cc/ML_ETL_Config.py")
from ML_ETL_Config import *
utils_module = MlEtlUtils(spark, sc)
to_import = [name for name in dir(utils_module) if not (name.startswith('_') | (name=="spark") | (name=="sc"))]
globals().update({name: getattr(utils_module, name) for name in to_import})
from pyspark.sql.functions import *
from pyspark.sql import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
pce_data = spark.read.parquet('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/pce_profile_past_role_data_17-09-2024')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
pce_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- mvp_company_name: string (nullable = true)
 |-- publicLink: string (nullable = true)
 |-- fullName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ethnicity: struct (nullable = true)
 |    |-- category: string (nullable = true)
 |    |-- probability: double (nullable = true)
 |-- functional_workload: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- business_function: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- msa: string (nullable = true)
 |-- experience_in_current_company: double (nullable = true)
 |-- experience_in_current_job_role: double (nullable = true)
 |-- years_of_experience: double (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- languageName: string (nullable = true)
 |    |    |-- proficiency: string (nullable = true)
 |-- country: string (nullable = true)
 |-- curr_company: string (nullable = true)
 |-- current

### Role input file

In [9]:
role_df = spark.read.csv('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/platform_role_india_23_09_2024.csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
role_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Platform Roles']

In [18]:
role_df.show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|Platform Roles     |
+-------------------+
|Data Annotator     |
|Data Entry Operator|
|Content Writer     |
|Copywriter         |
|Data Analyst       |
|Content Curator    |
|Content Editor     |
|Content Strategist |
+-------------------+

In [11]:
pce_df1 = pce_data.withColumn('Job Role',explode(col('current_job_role')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#role_df = pce_df1.filter(col('Job Role') == 'Product Marketing Manager')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
join_df = pce_df1.join(role_df, pce_df1['Job Role'] == role_df['Platform Roles'], 'inner').drop('Platform Roles')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### mas input file

In [21]:
loc_df=spark.read.csv('s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/india_msa_23_09_2024.csv',header=True).withColumnRenamed('msa', 'msa_data')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
loc_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['msa_data', 'Country', 'Region']

In [23]:
loc_df[['Country']].show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|Country|
+-------+
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
|India  |
+-------+
only showing top 20 rows

In [15]:
main_df = join_df.join(loc_df, join_df['msa'] == loc_df['msa_data'], 'inner').drop('msa_data')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
main_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mvp_company_name', 'publicLink', 'fullName', 'gender', 'ethnicity', 'functional_workload', 'business_function', 'msa', 'experience_in_current_company', 'experience_in_current_job_role', 'years_of_experience', 'languages', 'country', 'curr_company', 'current_industry', 'current_job_title', 'current_job_role', 'current_msa', 'current_synon_location', 'current_StartDate', 'current_EndDate', 'startDateYear', 'endDateYear', 'past_company', 'past_industry', 'past_company_experience', 'past_startDateYear', 'past_endDateYear', 'past_title', 'past_msa', 'past_synon_location', 'Job Role', 'Country', 'Region']

In [17]:
main_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

97854

### Inbount Output

In [20]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=main_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_migration_india_Inbount_data_23-09-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'GKWATXF0MQ63B222', 'HostId': 'NjZllV3XPXtsuj5c8PduqKUBl1APr7zy2YB0aSsWv1rLUSCiXQNEJJkSVCGsyMxnqvdRNm0UqUlHpFhslHPS6d0MuyfeCE5y2wjqTc0J8dY=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'NjZllV3XPXtsuj5c8PduqKUBl1APr7zy2YB0aSsWv1rLUSCiXQNEJJkSVCGsyMxnqvdRNm0UqUlHpFhslHPS6d0MuyfeCE5y2wjqTc0J8dY=', 'x-amz-request-id': 'GKWATXF0MQ63B222', 'date': 'Mon, 23 Sep 2024 05:05:04 GMT', 'x-amz-expiration': 'expiry-date="Sun, 29 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"293f1af61dd42e12bef86910ee856c41"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Sun, 29 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"293f1af61dd42e12bef86910ee856c41"', 'ServerSideEncryption': 'AES256'}

### Outbount 

In [15]:
main_df1 = role_df.join(loc_df,role_df['past_msa'] == loc_df['msa_data'],'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
main_df1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

398

In [17]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=main_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/pce_migration_canada_Outbount_data_20-09-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'MGQESVVFCXNTBXQF', 'HostId': 'oWAJaPVws2Yct6lBeh0hBhxbIhceTB5VP85Tad1ab7c0j9Dtk8JpuRcCMg04jLiYZp4sJVverBUvkdptgz3+cA==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'oWAJaPVws2Yct6lBeh0hBhxbIhceTB5VP85Tad1ab7c0j9Dtk8JpuRcCMg04jLiYZp4sJVverBUvkdptgz3+cA==', 'x-amz-request-id': 'MGQESVVFCXNTBXQF', 'date': 'Fri, 20 Sep 2024 04:21:09 GMT', 'x-amz-expiration': 'expiry-date="Thu, 26 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"62a0f2fba74fc40a08492479accc3ebd"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 26 Sep 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"62a0f2fba74fc40a08492479accc3ebd"', 'ServerSideEncryption': 'AES256'}